In [1]:
import re
import json
import requests
from bs4 import BeautifulSoup


URL = "https://ligafutsal.com.br/equipes"


def get_url(text):
    return re.search("(?P<url>https?://[^\s'\"]+)", text).group("url")

def main():
    r = requests.get(url=URL)
    data = r.content
    soup = BeautifulSoup(data, 'html.parser')
    equipes = []
    for equipe in soup.select('div.team-item > a'):
        dEquipe = {
            "logo": equipe.div.img["src"],
            "nomeCurto": equipe.find_all('p')[0].get_text(),
            "nome": equipe.find_all('p')[1].get_text(),
        }
        time = extrair_time(dEquipe["nomeCurto"])
        dEquipe["atletas"] = time
        equipes.append(
            dEquipe
        )
    with open('equipes.json', 'w') as fp:
        json.dump(equipes, fp)

def extrair_time(time):
    r = requests.get(url=URL + "/" + time)
    data = r.content
    soup = BeautifulSoup(data, 'html.parser')
    posicoes = soup.select(
        'ul.filter-athletes.small-12.medium-12.large-12.float-left > li')
    atletas = []
    for atleta in soup.select('ul.athletes-list.small-12.medium-12.large-12.float-left > li'):
        dAtleta = {
                "posicao": posicoes[int(atleta['position'])-1].get_text(),
                "urlAtleta": atleta.a["href"],
                "nomeCompleto": atleta.a["href"].replace('-', ' ').replace('https://ligafutsal.com.br/atletas/', '').replace('/', ''),
                "urlFoto": get_url(atleta.a.div["style"]),
                "nome": atleta.find_all('div')[1].find_all('p')[0].get_text(),
                "time": time
            }
        atletas.append(
            dAtleta
        )
    return atletas

   # print(re.findall(r'<ul class="filter\-athletes',data))
main()
